In [15]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
#from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
from imageio import imread
from PIL import Image
import numpy

In [31]:
img = imread(image)
img = imresize(img, (256, 256))   #256x256x3  
img = numpy.array(Image.fromarray(img).resize((256,256)))
img = img.transpose(2, 0, 1)   #channels first
img = img / 255.
img = torch.FloatTensor(img).cpu()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
transform = transforms.Compose([normalize])
img = transform(img)  # (3, 256, 256)
return img

NameError: name 'image' is not defined

In [32]:
def process_image(image):
    img = imread(image)
    img = numpy.array(Image.fromarray(img).resize((256,256)))
    img = img.transpose(2, 0, 1)   #channels first
    img = img / 255.
    img = torch.FloatTensor(img).cpu()
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    img = transform(img)  # (3, 256, 256)
    return img

In [24]:
# Define model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # in_channels = 3, out_channels = 6, filter_size = 5, 0 padding, stride 1
        self.conv1 = nn.Conv2d(3, 6, 5)    
        self.pool = nn.MaxPool2d(2, 2)     
        self.conv2 = nn.Conv2d(6, 12, 5)   
        self.fc1 = nn.Linear(12 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # consider an image of size 256x256
        x = F.relu(self.conv1(x))                # output size = [ (256 - 5 + 2(0) ) / 1 ] + 1 --> 252x252
        x = self.pool(x)                         # output_size = 252 / 2 --> 126x126
        x = F.relu(self.conv2(x))                # output size = [ (126 - 5 + 2(0) ) / 1 ] + 1 --> 122x122
        x = self.pool(x)                         # output size = 122/2 --> 61x61
        x = x.view(-1, 12 * 61 * 61)             # (1, 44652)
        x = F.relu(self.fc1(x))                  # (1, 120)
        x = self.fc2(x)                          # (1, 10)
        return x

In [25]:
# Initialize model
model = CNN()
model = model.cpu()
# Initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [33]:

image = process_image(r'N:\deep learning\dog.jpg')
image = image.unsqueeze(0)  

In [34]:
output = model(image)
print(output.shape)

torch.Size([1, 10])


In [35]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([12, 6, 5, 5])
conv2.bias 	 torch.Size([12])
fc1.weight 	 torch.Size([120, 44652])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([10, 120])
fc2.bias 	 torch.Size([10])


In [36]:
torch.save(model.state_dict(), 'model.pth.tar')

In [37]:
#model = CNN()
model.load_state_dict(torch.load('model.pth.tar'))
model.eval() 

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [38]:
model = CNN().cpu()

In [39]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [40]:
checkpoint = {'epoch': 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': 0.2}
torch.save(checkpoint, 'model.pth.tar')

In [41]:
checkpoint = torch.load('model.pth.tar')

In [42]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [43]:

# If testing
model.eval()
# If resume training
model.train()

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44652, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [44]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.0701, -0.1054,  0.0735,  0.0739,  0.0386],
                        [-0.0842,  0.0391, -0.0419,  0.1060, -0.1036],
                        [-0.0052, -0.0690, -0.0727, -0.0530, -0.0938],
                        [ 0.1114,  0.1123, -0.1049,  0.0681,  0.0098],
                        [ 0.0034, -0.0317,  0.0251,  0.0086,  0.0066]],
              
                       [[-0.0111, -0.0576,  0.0101,  0.0039, -0.0112],
                        [ 0.0107,  0.0233,  0.0112, -0.0692, -0.0411],
                        [-0.0367, -0.1081, -0.0464,  0.0884, -0.0355],
                        [ 0.1092, -0.0350,  0.0752, -0.0199, -0.0443],
                        [-0.0806, -0.0706,  0.0458, -0.0593,  0.0737]],
              
                       [[ 0.0474, -0.0194, -0.1104,  0.0978, -0.0270],
                        [-0.0706,  0.0987, -0.1011,  0.0515, -0.0497],
                        [ 0.0648, -0.0638, -0.0246,  0.0319,  0.0193],
               